In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
timers = []
timers_import_start = time.time() 
# Trainer (has labels)
#trainer_df = pd.read_csv('../datasets/cleaned_SOLIDtest6K_trainer.tsv', sep="\t")
# learner_tweets_df = pd.read_csv('datasets/cleaned_OLID.tsv', sep="\t")
learner_tweets_df = pd.read_csv('../datasets/cleaned_hatespeech_offensive.tsv', sep="\t")

timers_import_end = time.time()
print(timers_import_end - timers_import_start)

0.024083375930786133


In [23]:
import numpy as np
# trainer_labels = trainer_df['label'].values
# trainer_tweets = trainer_df['tweet'].values
#learner_tweets_df['labels'] = learner_tweets_df['average'].apply(lambda x: 1 if x >= 0.8 else 0) # threshold the average values
# learner_tweets_df['subtask_a'] = learner_tweets_df['subtask_a'].map({'OFF': 1, 'NOT': 0})

# learner_tweets_df['labels'] = learner_tweets_df["subtask_a"]
# learner_tweets_df.drop(["subtask_a","subtask_b","subtask_c"], axis=1, inplace=True)
# sample_size = 40000
positive_ratio = 0.5

# Select the most confident positive values
#semi_tweets_pos_df = learner_tweets_df[learner_tweets_df['average'] > 0.8].sample(n=np.floor(sample_size*positive_ratio).astype(int), random_state=1)

# Select the most confident negative values
#semi_tweets_neg_df = learner_tweets_df[learner_tweets_df['average'] < 0.3].sample(n=np.floor(sample_size*(1-positive_ratio)).astype(int), random_state=1)

#semi_tweets_df = pd.concat([semi_tweets_pos_df, semi_tweets_neg_df])

# Count the number of positive and negative samples
positive_samples = learner_tweets_df[learner_tweets_df['label'] == 1]
negative_samples = learner_tweets_df[learner_tweets_df['label'] == 0]

# Determine the minimum number of samples between the two classes
min_samples = min(len(positive_samples), len(negative_samples))

# Sample an equal number of positive and negative samples
balanced_samples = pd.concat([
    positive_samples.sample(n=min_samples, random_state=42),
    negative_samples.sample(n=min_samples, random_state=42)
])

# Update the semi_tweets_df with the balanced samples
semi_tweets_df = balanced_samples

semi_tweets_df = semi_tweets_df.sample(frac=1, random_state=42)


semi_tweets = semi_tweets_df['tweet'].values
semi_labels = semi_tweets_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(semi_tweets, semi_labels, test_size=0.2)



In [24]:
semi_tweets_df.head(10)

,id,label,tweet
20495,20941,0,real talk not being byes or nothin the whole n...
15840,16209,1,rt joebudden women all over america are overst...
16704,17089,1,rt nothinglikeit these hoes getting dressed up...
2251,2296,1,touchdown with on the clock michigan some hoes
18559,18968,0,rt amyalkon school admins take hungry kids lun...
9475,9735,0,going up to the view point and seeing star chi...
5012,5161,0,toddirwin nfl forgetabout they got until febru...
18011,18415,1,rt woridstarhlphop when a bad bitch asks you t...
13743,14079,0,overripe eggplants are not good eats uncletweet
4191,4314,1,mr_lewis_ brookebrooklyn_ ohh yaa he wants bro...


In [29]:
import torch
from torch.utils.data import DataLoader, Dataset


class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
learner_dataset = TweetDataset(X_train, y_train)


# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
learner_loader = DataLoader(learner_dataset, batch_size=6, shuffle=False)


In [30]:
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm
from transformers import AutoTokenizer, ConvBertModel, ConvBertForSequenceClassification

# Number of epochs
num_epochs = 1

tokenizer = AutoTokenizer.from_pretrained("YituTech/conv-bert-base")


# Initialize the model
model = ConvBertForSequenceClassification.from_pretrained("YituTech/conv-bert-base", num_labels=2)


# Set up the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Check if CUDA is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at YituTech/conv-bert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ConvBertForSequenceClassification(
  (convbert): ConvBertModel(
    (embeddings): ConvBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ConvBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x ConvBertLayer(
          (attention): ConvBertAttention(
            (self): ConvBertSelfAttention(
              (query): Linear(in_features=768, out_features=384, bias=True)
              (key): Linear(in_features=768, out_features=384, bias=True)
              (value): Linear(in_features=768, out_features=384, bias=True)
              (key_conv_attn_layer): SeparableConv1D(
                (depthwise): Conv1d(768, 768, kernel_size=(9,), stride=(1,), padding=(4,), groups=768, bias=False)
                (pointwise): Conv1d(7

In [31]:
model.train()
num_epochs = 1
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    semi_labeled_dataloader_iterator = iter(learner_loader)
    num_batches = len(learner_loader)
    # num_batches = 5000
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(semi_labeled_dataloader_iterator, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = model(**inputs)
            logits = outputs.logits
    
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

  0%|                                                                                         | 0/1110 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\mdoga\AppData\Local\Temp\ipykernel_18768\692897503.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████████████████████████████████████████████████████████████████████████| 1110/1110 [01:07<00:00, 16.54it/s]


In [46]:
class TweetDatasetTest(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)
test_dataset = TweetDatasetTest(test_encodings, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=12, shuffle=False)


In [47]:

model.eval()
correct = 0
total = 1
num_epochs = 1
prediction_list = np.array([])


with torch.no_grad():
    test_start  = time.time()
    print('start')
    for batch in test_loader:
        #print(batch)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
        prediction_list = np.append(prediction_list, predictions.detach().cpu().numpy())
    print('end')
    
    test_end = time.time()


accuracy = correct / total
print(f'Validation Accuracy: {accuracy}')

start
end
Validation Accuracy: 0.9562087582483503


In [48]:
from sklearn.metrics import classification_report

# Assuming you have the true labels in `val_labels` and the predicted labels in `prediction_list`
report = classification_report(y_test, prediction_list)

print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       851
           1       0.97      0.94      0.96       815

    accuracy                           0.96      1666
   macro avg       0.96      0.96      0.96      1666
weighted avg       0.96      0.96      0.96      1666



In [9]:
from sklearn.metrics import confusion_matrix

# Assuming you have the true labels in `val_labels` and the predicted labels in `prediction_list`
cm = confusion_matrix(y_test, prediction_list)

# Extract TP, TN, FP, FN from the confusion matrix
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]

print(f"True Positives (TP): {TP}")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")


True Positives (TP): 782
True Negatives (TN): 826
False Positives (FP): 13
False Negatives (FN): 45


In [21]:
model.save_pretrained('models/HSO_BERT_2')